# 3: Assessing model performance and applicability

Author: Daniel Lusk

## Imports

In [ ]:
import pandas as pd
import dask_geopandas as dgpd
from utils.models import TrainedSet, Prediction

%load_ext autoreload
%autoreload 2

## Model performance

Get the training run data from the results dataframe and choose a model

In [ ]:
# Load a model
results_df = pd.read_csv("./results/training_results.csv").copy()
training_id = "2023-07-21_17-01-49"
results = results_df.loc[(results_df["Run ID"] == training_id)].sort_values(by="Test r-squared", ascending=False)
row = results.iloc[0]
trained = TrainedSet.from_results_row(row)

Load unseen predictor data and run prediction

In [ ]:
# Load new high-res data
new_df = dgpd.read_parquet(
    "data/collections/tiled_5x5_deg_MOD09GA.061_ISRIC_soil_WC_BIO_VODCA_0.01_deg/tile_95_35_100_40.parq"
)

pred = Prediction(trained, new_df.compute())

Plot the masked (by AoA) trait predictions

In [ ]:
pred.df.plot(column=pred.df.columns[-2], markersize=0.5)